<a href="https://colab.research.google.com/github/graylan0/quantum-machine-learning/blob/main/gpt3.5.turbo.barkai.sentiment.voice.adapter.voice.tts.multithreaded.smallmodels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --pre torch torchvision -f https://download.pytorch.org/whl/nightly/cu102/torch_nightly.html -U
!pip install git+https://github.com/suno-ai/bark.git
!pip install openai


Looking in links: https://download.pytorch.org/whl/nightly/cu102/torch_nightly.html
  Cloning https://github.com/suno-ai/bark.git to /tmp/pip-req-build-n56xa8fe
  Running command git clone --filter=blob:none --quiet https://github.com/suno-ai/bark.git /tmp/pip-req-build-n56xa8fe
  Resolved https://github.com/suno-ai/bark.git to commit 56b0ba13f7c281cbffa07ea9abf7b30273a60b6a
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 23.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 60.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 75.3 MB/s eta 0:00:00
    

In [12]:
import numpy as np
import uuid
from scipy.io.wavfile import write as write_wav
import openai
from bark import generate_audio, SAMPLE_RATE
import os
from concurrent.futures import ThreadPoolExecutor

os.environ["SUNO_USE_SMALL_MODELS"] = "True"
os.environ["SUNO_OFFLOAD_CPU"] = "True"

# Initialize OpenAI API with your key
openai_api_key = "getopenaikeyfromhereandputhere;https://platform.openai.com/account/api-keys"
openai.api_key = openai_api_key

def analyze_and_split_text(input_text):
    # Use GPT-3.5 turbo to analyze the input text, infer emotions, and split it into meaningful segments
    response = openai.ChatCompletion.create(
        model='gpt-3.5-turbo',
        messages=[
            {'role': 'system', 'content': 'You are an expert in analyzing text and inferring emotions. Analyze the following text and split it into meaningful segments, providing the emotions for each segment. Return the segments in the following format: "Segment: [text] - Emotion: [emotion]". Consider the following special features: [laughter] for laughter, [sighs] for sighs, [music] for music, [gasps] for gasps, [clears throat] for clearing throat, ... for hesitations, ♪ for song lyrics, CAPITALIZATION for emphasis, [MAN] for male speakers, and [WOMAN] for female speakers.'},
            {'role': 'user', 'content': input_text}
        ],
    )
    return response['choices'][0]['message']['content'].split('\n')

def generate_audio_with_emotion(segment, emotion):
    # Map emotions to Bark's special tags
    emotion_to_tag = {
        'laughter': '[laughter]',
        'joy': '[laughs]',
        'sadness': '[sighs]',
        'music': '[music]',
        'surprise': '[gasps]',
        'hesitation': '...',
        'song': '♪',
        'emphasis': 'CAPITALIZATION',
        'male': '[MAN]',
        'female': '[WOMAN]'
    }

    # Apply the corresponding tag based on the detected emotion
    tag = emotion_to_tag.get(emotion.lower())
    if tag:
        segment = tag + ' ' + segment

    # Generate audio using Bark with the modified text
    audio_array = generate_audio(segment, history_prompt="v2/en_speaker_6")
    return audio_array

def process_segment(segment_analysis):
    try:
        segment, emotion = segment_analysis.split(' - Emotion: ')
        segment = segment.replace('Segment: ', '').strip()
        audio_array = generate_audio_with_emotion(segment, emotion.strip())
        silence = np.zeros(int(0.75 * SAMPLE_RATE))  # quarter second of silence
        return [audio_array, silence.copy()]
    except ValueError:
        print(f"Error processing segment: {segment_analysis}")
        return []

def generate_response(message):
    # Analyze the text for emotions and split it into meaningful segments
    segments_analysis = analyze_and_split_text(message)

    # Process each segment and generate audio using threading with max 2 threads
    pieces = []
    with ThreadPoolExecutor(max_workers=14) as executor:
        results = list(executor.map(process_segment, segments_analysis))
        for result in results:
            pieces += result

    # Concatenate all audio pieces
    audio = np.concatenate(pieces)

    # Generate a random file name
    wav_file_name = str(uuid.uuid4()) + ".wav"

    # Save the audio to a WAV file in the current directory
    write_wav(wav_file_name, SAMPLE_RATE, audio)

    print(f"Audio file generated: {wav_file_name}")

# Test the function with a message
generate_response("I PITY DA FOOL ")




100%|██████████| 9/9 [00:08<00:00,  1.08it/s]


Audio file generated: 496e1584-b9eb-4ab2-a893-38a2142414b6.wav
